In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
121359,Thiago Emiliano da Silva,11,2,90,Chelsea,Burnley,1.963440,0.320009,True,2,...,0.0,1.0,0.0,73.0,79.0,92.4,67.0,0.0,0.0,1
121360,Thomas Partey,11,3,0,Arsenal,Watford,1.241053,0.320449,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
121361,Timothy Castagne,11,2,90,Leicester City,Leeds,1.207136,0.569761,False,2,...,0.0,0.0,0.0,27.0,37.0,73.0,27.0,1.0,2.0,1
121362,Timo Werner,11,4,0,Chelsea,Burnley,1.963440,0.320009,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
121363,Tim Krul,11,1,90,Norwich,Brentford,0.410638,0.440370,False,4,...,0.0,0.0,0.0,15.0,33.0,45.5,12.0,0.0,0.0,1
121364,Tino Livramento,11,2,90,Southampton,Aston Villa,0.562053,0.909812,True,6,...,0.0,2.0,0.0,41.0,48.0,85.4,45.0,0.0,0.0,1
121365,Tobi Omole,11,2,0,Tottenham Hotspur,Everton,1.535073,0.944279,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
121366,Toby Alderweireld,11,2,0,Tottenham Hotspur,Everton,1.535073,0.944279,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
121367,Todd Cantwell,11,3,0,Norwich,Brentford,0.410638,0.440370,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
121368,Tomas Soucek,11,3,90,West Ham United,Liverpool,0.780200,1.908380,True,1,...,0.0,0.0,0.0,28.0,37.0,75.7,15.0,0.0,0.0,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

12

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Eric Bailly,12,2,0,Manchester United,Watford,2.006938,0.320449,False,0,...,10663,1,4,0.0,-366,1038,1404,0,2021-11-20T15:00:00Z,2122
1,Keinan Davis,12,4,0,Aston Villa,Brighton and Hove Albion,0.909812,0.566017,True,0,...,236157,0,2,0.0,35746,49856,14110,0,2021-11-20T15:00:00Z,2122
2,Ayotomiwa Dele-Bashiru,12,3,0,Watford,Manchester United,0.320449,2.006938,True,0,...,1133,1,4,0.0,-44,0,44,0,2021-11-20T15:00:00Z,2122
3,James Ward-Prowse,12,3,90,Southampton,Norwich,0.562053,0.410638,False,2,...,180183,1,2,18.0,-2204,10161,12365,0,2021-11-20T15:00:00Z,2122
4,Bruno Miguel Borges Fernandes,12,3,90,Manchester United,Watford,2.006938,0.320449,False,2,...,1592480,1,4,5.0,-47341,29779,77120,0,2021-11-20T15:00:00Z,2122
5,Adrián San Miguel del Castillo,12,1,0,Liverpool,Arsenal,1.908380,1.241053,True,0,...,18279,0,4,0.0,-353,139,492,0,2021-11-20T17:30:00Z,2122
6,James McArthur,12,3,0,Crystal Palace,Burnley,0.549389,0.320009,False,0,...,294259,3,3,0.0,-2102,35378,37480,0,2021-11-20T15:00:00Z,2122
7,Moise Kean,12,4,0,Everton,Manchester City,0.944279,2.378593,False,0,...,1165,0,3,0.0,-12,0,12,0,2021-11-21T14:00:00Z,2122
8,Kwadwo Baah,12,3,0,Watford,Manchester United,0.320449,2.006938,True,0,...,8025,1,4,0.0,-1069,188,1257,0,2021-11-20T15:00:00Z,2122
9,Paul Dummett,12,2,0,Newcastle United,Brentford,0.540469,0.440370,True,0,...,854,3,3,0.0,-2,45,47,0,2021-11-20T15:00:00Z,2122


In [10]:
df_latest.shape

(631, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,Kasper Schmeichel,Leicester City,1.00
1,Jonny Evans,Leicester City,1.00
2,Marc Albrighton,Leicester City,0.75
3,Nampalys Mendy,Leicester City,0.00
4,Danny Ward,Leicester City,1.00


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Antonio Rüdiger
Jorginho
Thiago Silva
Andreas Christensen
Mateo Kovačić
Reece James
Kai Havertz
Trevoh Chalobah
César Azpilicueta
Romelu Lukaku
Marcos Alonso
Callum Hudson-Odoi
Ben Chilwell
Mason Mount
N'Golo Kanté
Timo Werner
Ruben Loftus-Cheek
Hakim Ziyech
Christian Pulisic
Ross Barkley
Kepa Arrizabalaga
Malang Sarr
Saúl Ñíguez
Emerson Palmieri
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
João Cancelo
Ederson
Bernardo Silva
Rúben Dias
Rodri
Kyle Walker
Gabriel Jesus
Aymeric Laporte
İlkay Gündoğan
Jack Grealish
Phil Foden
Raheem Sterling
Kevin De Bruyne
Ferrán Torres
Riyad Mahrez
John Stones
Nathan Aké
Fernandinho
Cole Palmer
Zack Steffen
Benjamin Mendy
Oleksandr Zinchenko
James Mcatee
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Virgil van Dijk
Mohamed Salah
Sadio Mané
Alisson
Trent Alexander-Arnold
Jordan

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Edouard Mendy,2021-08-14,Chelsea,Premier League,1,0,0,20,0,...,0.0,0,0,18,20,90.0,10,0,0,Edouard Mendy
1,2021-2022,Edouard Mendy,2021-08-22,Chelsea,Premier League,2,0,0,46,0,...,0.0,0,0,37,43,86.0,30,0,0,Edouard Mendy
2,2021-2022,Edouard Mendy,2021-08-28,Chelsea,Premier League,3,0,0,47,0,...,0.0,0,0,26,40,65.0,9,0,0,Edouard Mendy
3,2021-2022,Edouard Mendy,2021-09-11,Chelsea,Premier League,4,0,0,42,1,...,0.0,0,0,30,36,83.3,17,0,0,Edouard Mendy
4,2021-2022,Edouard Mendy,2021-09-25,Chelsea,Premier League,6,0,0,38,0,...,0.0,0,0,28,34,82.4,16,0,0,Edouard Mendy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,2021-2022,Jeff Hendrick,2021-10-02,Newcastle United,Premier League,7,1,1,27,15,...,0.0,0,0,19,22,86.4,21,1,1,Jeff Hendrick
3576,2021-2022,Dwight Gayle,2021-09-25,Newcastle United,Premier League,6,0,0,3,4,...,0.0,0,0,2,3,66.7,3,0,0,Dwight Gayle
3577,2021-2022,Dwight Gayle,2021-10-02,Newcastle United,Premier League,7,1,0,3,4,...,0.0,0,0,0,1,0.0,2,0,0,Dwight Gayle
3578,2021-2022,Dwight Gayle,2021-11-06,Newcastle United,Premier League,11,0,0,1,1,...,0.0,0,0,0,0,,0,0,0,Dwight Gayle


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['James Mcatee']

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['James Mcatee'] = 'James McAtee'
# match_list['Przemysław Płacheta'] = 'Przemyslaw Placheta'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [30]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
# df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

In [41]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [42]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
119145,Harry Kane,8,4,90,Tottenham Hotspur,Newcastle United,1.535970,0.540785,False,12,...,0.8,3,1,23,25,92,20,1,2,1
119762,Harry Kane,9,4,90,Tottenham Hotspur,West Ham United,1.535970,0.780656,False,2,...,0,1,0,21,26,80.8,24,2,4,1
120381,Harry Kane,10,4,90,Tottenham Hotspur,Manchester United,1.535124,2.007004,True,2,...,0,0,0,15,26,57.7,17,0,0,1
121004,Harry Kane,11,4,90,Tottenham Hotspur,Everton,1.535073,0.944279,False,2,...,0.3,3,0,26,35,74.3,32,1,2,1
121926,Harry Kane,12,4,90,Tottenham Hotspur,Leeds,1.535073,0.569761,True,2,...,0.0,2,1,20,24,83.3,32,3,6,1


In [43]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
122033,N'Golo Kanté,12,3,90,Chelsea,Leicester City,1.963440,1.207136,False,9,...,0.0,3,1,59,66,89.4,52,0,0,1
122034,Josh Martin,12,3,0,Norwich,Southampton,0.410638,0.562053,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
122035,Wilfred Ndidi,12,3,90,Leicester City,Chelsea,1.207136,1.963440,True,2,...,0.0,0,0,35,51,68.6,33,1,3,1
122036,Matt Ritchie,12,2,90,Newcastle United,Brentford,0.540469,0.440370,True,4,...,0.2,3,1,44,63,69.8,30,1,1,1
122037,Nathan Redmond,12,3,0,Southampton,Norwich,0.562053,0.410638,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
122038,Mathew Ryan,12,1,0,Brighton and Hove Albion,Aston Villa,0.566017,0.909812,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
122039,Ryan Fredericks,12,2,0,West Ham United,Wolverhampton Wanderers,0.780200,0.845942,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [44]:
df_latest_further.shape

(631, 57)

In [45]:
df_train_new.shape

(122040, 59)

In [46]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [47]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [48]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
120,Arsenal,Newcastle United,2021-11-27T12:30:00Z,13,0
121,Crystal Palace,Aston Villa,2021-11-27T15:00:00Z,13,1
122,Liverpool,Southampton,2021-11-27T15:00:00Z,13,2
123,Norwich,Wolverhampton Wanderers,2021-11-27T15:00:00Z,13,3
124,Brighton and Hove Albion,Leeds,2021-11-27T17:30:00Z,13,4
125,Brentford,Everton,2021-11-28T14:00:00Z,13,5
126,Burnley,Tottenham Hotspur,2021-11-28T14:00:00Z,13,6
127,Leicester City,Watford,2021-11-28T14:00:00Z,13,7
128,Manchester City,West Ham United,2021-11-28T14:00:00Z,13,8
129,Chelsea,Manchester United,2021-11-28T16:30:00Z,13,9


In [49]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [50]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,3,4
1,13,1,Crystal Palace,Aston Villa,2021-11-27T15:00:00Z,31,7
2,13,2,Liverpool,Southampton,2021-11-27T15:00:00Z,14,20
3,13,3,Norwich,Wolverhampton Wanderers,2021-11-27T15:00:00Z,45,39
4,13,4,Brighton and Hove Albion,Leeds,2021-11-27T17:30:00Z,36,2


In [51]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [52]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
6,Granit Xhaka,0,3,7,48,3,Xhaka
8,Héctor Bellerín,0,2,9,48,3,Bellerín
10,Sead Kolasinac,0,2,11,43,3,Kolasinac
...,...,...,...,...,...,...,...
615,Owen Otasowie,0,3,442,45,39,Otasowie
617,Andreas Söndergaard,0,1,444,40,39,Söndergaard
619,Yerson Mosquera Valdelamar,0,2,446,45,39,Mosquera
624,Patrick Cutrone,0,4,523,50,39,Cutrone


In [53]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [54]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [55]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,13,0,Newcastle United,Arsenal,2021-11-27T12:30:00Z,Joseph Willock,3,58,100,Willock,False
1,13,0,Newcastle United,Arsenal,2021-11-27T12:30:00Z,Jonjo Shelvey,3,53,100,Shelvey,False
2,13,0,Newcastle United,Arsenal,2021-11-27T12:30:00Z,Matt Ritchie,2,49,None,Ritchie,False
3,13,0,Newcastle United,Arsenal,2021-11-27T12:30:00Z,Ciaran Clark,2,44,100,Clark,False
4,13,0,Newcastle United,Arsenal,2021-11-27T12:30:00Z,Karl Darlow,1,45,100,Darlow,False


In [56]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Bernd Leno,1,46,None,Leno,True
1,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Rúnar Alex Rúnarsson,1,40,0,Rúnarsson,True
2,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Willian Borges Da Silva,3,63,0,Willian,True
3,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Pierre-Emerick Aubameyang,4,99,100,Aubameyang,True
4,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Cédric Soares,2,42,None,Cédric,True


In [57]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [58]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [59]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Bernd Leno,1,4.6,1.0,Leno,True,1.241053,0.540469,2122
1,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Rúnar Alex Rúnarsson,1,4.0,0.0,Rúnarsson,True,1.241053,0.540469,2122
2,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Willian Borges Da Silva,3,6.3,0.0,Willian,True,1.241053,0.540469,2122
3,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Pierre-Emerick Aubameyang,4,9.9,1.0,Aubameyang,True,1.241053,0.540469,2122
4,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Cédric Soares,2,4.2,1.0,Cédric,True,1.241053,0.540469,2122
5,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Alexandre Lacazette,4,8.3,1.0,Lacazette,True,1.241053,0.540469,2122
6,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Granit Xhaka,3,4.8,0.0,Xhaka,True,1.241053,0.540469,2122
7,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Pablo Marí,2,4.2,1.0,Marí,True,1.241053,0.540469,2122
8,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Héctor Bellerín,2,4.8,0.0,Bellerín,True,1.241053,0.540469,2122
9,13,0,Arsenal,Newcastle United,2021-11-27T12:30:00Z,Calum Chambers,2,4.2,1.0,Chambers,True,1.241053,0.540469,2122


In [60]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')